In [1]:
import pandas as pd
from sodapy import Socrata

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
client = Socrata("data.seattle.gov", None)

In [3]:
results = client.get("tmmm-ytt6", limit=8000000)

In [4]:
results_df = pd.DataFrame.from_records(results)

In [5]:
results_df.shape

(8000000, 11)

In [6]:
results_df['checkoutyear'].unique()

array(['2009', '2017', '2010', '2011', '2012', '2018'], dtype=object)

In [10]:
results_df.to_csv('/home/jupyter/spl_checkouts.csv')